In [11]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [90]:
# initially from https://towardsdatascience.com/online-experiments-tricks-variance-reduction-291b6032dcd7,
# but simplified more

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [91]:
size = 10000
treatment_effect = 1.0
df = pd.DataFrame({"y": np.random.normal(loc=0, scale=1, size=size)})

# create a covariate that's correlated with y
df["x"] = df["y"] + np.random.normal(loc=0, scale=0.1, size=size)

# another covariate that's correlated with y
df["x1"] = df["y"] * 0.3

# random assign rows to two groups 0 and 1, control and experiment
df["group"] = np.random.randint(0, 2, df.shape[0])

# for treatment group add a treatment effect
df.loc[df["group"] == 1, "y"] += treatment_effect

In [93]:
theta = df.cov()["x"]["y"] / df.cov()["x"]["x"]
df["y_cuped"] = df.y - theta * (df.x - df.x.mean())  # cuped

print(f"variance = {df['y'].var() * ( 1- df.corr()['x']['y']**2)}")

# Notice that y_cuped is unbiased and has lower variance
df[["y", "y_cuped", "group"]].groupby("group").agg(["mean", "var"])

variance = 0.25827498611793215


y             y_cuped          
           mean       var      mean       var
group                                        
0      0.051162  1.028873  0.031477  0.009680
1      1.008550  0.995125  1.028039  0.010263

Equivalently we can fit a linear regression and go from there.

In [84]:
df_group_0 = df.loc[df.group == 0, :]

xhat = LinearRegression().fit(df_group_0[["x"]], df_group_0["y"]).predict(df[["x"]])
df["y_cuped_linear"] = df.y - (xhat - np.mean(xhat))  # cuped

# Notice that y_cuped and y_cuped_linear are the same
df[["y", "y_cuped", "y_cuped_linear", "group"]].groupby("group").agg(["mean", "var"])

y             y_cuped           y_cuped_linear          
           mean       var      mean       var           mean       var
group                                                                 
0     -0.018593  1.014106 -0.003831  0.010120      -0.003940  0.010064
1      1.011296  0.966172  0.996445  0.009934       0.996554  0.009880

Essentially we are finding the best linear model that depends on covarariates $X_1, X_2, \ldots, X_p$ and predicts the response $Y$. Those covariates shouldn't depend on the treatment variable $Z$, i.e., how groups are split. 

However, there's nothing special about the linear model. We can use any model that depends on the covariates and predicts the response. For example, we can use a random forest. 

I think this is the difference between cuped and cupac. 

In [81]:
df_group_0 = df.loc[df.group == 0, :]
model = RandomForestRegressor().fit(df_group_0[["x"]], df_group_0["y"])

xhat = model.predict(df[["x"]])
df["y_cupac_tree"] = df.y - (xhat - np.mean(xhat))

# Notice that y_cuped_tree is unbiased and has the lowest variance
df[["y", "y_cuped", "y_cupac_tree", "group"]].groupby("group").agg(["mean", "var"])

y             y_cuped           y_cupac_tree          
           mean       var      mean       var         mean       var
group                                                               
0     -0.018593  1.014106 -0.003831  0.010120    -0.003614  0.002000
1      1.011296  0.966172  0.996445  0.009934     0.996226  0.014393

In [85]:
# we can also regress on more on than one variables
model = RandomForestRegressor().fit(
    df.loc[df.group == 0, ["x", "x1"]], df.loc[df.group == 0, "y"]
)
xhat = model.predict(df[["x", "x1"]])

df["y_cupac_tree_two"] = df.y - (xhat - np.mean(xhat))

# Notice that y_cuped_tree is unbiased and has the lowest variance
df[["y", "y_cuped", "y_cupac_tree", "y_cupac_tree_two", "group"]].groupby("group").agg(
    ["mean", "var"]
)

y             y_cuped           y_cupac_tree            \
           mean       var      mean       var         mean       var   
group                                                                  
0     -0.018593  1.014106 -0.003831  0.010120    -0.003614  0.002000   
1      1.011296  0.966172  0.996445  0.009934     0.996226  0.014393   

      y_cupac_tree_two            
                  mean       var  
group                             
0            -0.003727  0.000018  
1             0.996340  0.000048

What happens when the covariate depends on the treatment variable?


In [86]:
# another covariate that's correlated with y but also depends on treatment variable
df["x2"] = df["group"] + df["y"]

xhat = LinearRegression().fit(df[["x2"]], df["y"]).predict(df[["x2"]])
df["y_cuped_bad"] = df.y - (xhat - np.mean(xhat))  # cuped

# small difference between y_cuped and y_cuped_bad :O -- this is wrong, it would mean treatment doesn't have an effect.
df[["y", "y_cuped", "y_cuped_bad", "group"]].groupby("group").agg(["mean", "var"])

y             y_cuped           y_cuped_bad          
           mean       var      mean       var        mean       var
group                                                              
0     -0.018593  1.014106 -0.003831  0.010120    0.739110  0.063995
1      1.011296  0.966172  0.996445  0.009934    0.249033  0.060970